# 数据迁移

In [ ]:
import sqlite3 as sql
import pandas as pd

### 建立一些参数

In [ ]:
db_name = "NETWORK_OPS"
table_name = "DB_NETOPS"
excel_path = "excel/ops_info.xlsx"

## 连接（新建）数据库
* 同时生成一个游标

In [ ]:
DB = sql.connect(db_name)
CURSOR = DB.cursor()

## 在数据库里创建运维表

In [ ]:
init_sql = f'''CREATE TABLE IF NOT EXISTS {table_name} 
(id INTEGER PRIMARY KEY,设备类型 TEXT,设备IP TEXT, 登入用户名 TEXT, 登入密码 TEXT)'''
CURSOR.execute(init_sql)

## 从EXCEL文档读入数据

In [ ]:
datas = pd.read_excel(excel_path)
datas

In [ ]:
list(datas.values[0])

## 完成数据迁移

In [ ]:
for line_num in range(datas.shape[0]):
    #print(datas.values[line_num])
    db_line = list(datas.values[line_num])
    db_line.insert(0,line_num+1)
    print(db_line)
    CURSOR.execute(f"INSERT INTO {table_name} VALUES(?,?,?,?,?)", db_line)

## 检查迁移结果

In [ ]:
CURSOR.execute(f"SELECT * FROM {table_name}")
CURSOR.fetchall()

## 为数据库存盘

In [ ]:
DB.commit()

# 自动化运维

In [1]:
import netmiko
import os
import time
import sqlite3 as sql

## 从数据库读取数据

In [2]:
def get_device_info(db_name, table_name):
    DB = sql.connect(db_name)
    CURSOR = DB.cursor()
    CURSOR.execute(f"select * from {table_name}")
    return CURSOR.fetchall()
    
device_info = get_device_info("NETWORK_OPS", "DB_NETOPS") 

In [3]:
device_info

[(1, 'cisco_ios', '192.168.1.60', 'ciscouser', 'cisco@123'),
 (2, 'cisco_ios', '192.168.1.61', 'ciscouser', 'cisco@123'),
 (3, 'cisco_ios', '192.168.1.62', 'ciscouser', 'cisco@123'),
 (4, 'huawei', '192.168.1.70', 'yeslab', 'huawei@123'),
 (5, 'huawei', '192.168.1.71', 'yeslab', 'huawei@123'),
 (6, 'huawei', '192.168.1.72', 'yeslab', 'huawei@123'),
 (7, 'huawei', '192.168.1.73', 'yeslab', 'huawei@123'),
 (8, 'huawei', '192.168.1.74', 'yeslab', 'huawei@123'),
 (9, 'huawei', '192.168.1.75', 'yeslab', 'huawei@123')]

## 自动化备份函数

In [6]:
def create_dict(ori_infos):
    devices = []
    for info in ori_infos:
        device = {}
        device['device_type'] = info[1]
        device['port'] = 22
        device['ip'] = info[2]
        device['username'] = info[3]
        device['password'] = info[4]
        devices.append(device)
    return devices
    
def check_folder(device_path, path):
    if not os.path.isdir(path):
        os.mkdir(path)
    path += device_path + '/'
    if not os.path.isdir(path):
        os.mkdir(path)
        print("{} 目录已建立".format(path))
    return path

def create_filename():
    time_struct = time.localtime()
    str_year = str(time.localtime().tm_year) + "_"
    str_mon = str(time.localtime().tm_mon) + "_"
    str_mday = str(time.localtime().tm_mday) + "_"
    str_hour = str(time.localtime().tm_hour) + "_"
    str_min = str(time.localtime().tm_min) + "_"
    str_sec = str(time.localtime().tm_sec) + ".cfg"
    filename = str_year + str_mon + str_mday + str_hour + str_min + str_sec
    return filename

def save_config(device_type, ip, path, config_data):
    device_path = device_type + '_' + ip
    this_path = check_folder(device_path, path)
    filename = create_filename()
    full_path = this_path + filename
    with open(full_path, 'w') as f:
        f.write(config_data)
        
def backup(config_save_path="config/"):
    ori_infos = get_device_info("NETWORK_OPS", "DB_NETOPS")
    devices = create_dict(ori_infos)
    print("\n\\(˙<>˙)==[配置备份中，活让我干就好，您喝茶]\n")
    for device_info in devices:
        server = netmiko.ConnectHandler(**device_info)
        print("已登入设备：{}".format(device_info["ip"]))
        if device_info['device_type'] == 'cisco_ios':
            config = server.send_command("show running-config")
        elif device_info['device_type'] == 'huawei':
            config = server.send_command("display current-configuration")
        print("已获取配置")
        server.disconnect()
        del server
        save_config(device_info['device_type'], device_info["ip"], config_save_path, config)
        print("已保存配置")
    print("\n\\(˙ω˙)==[备份结束，新备份文件已经就位，请大哥检查]")

## 执行备份过程

In [8]:
backup()


\(˙<>˙)==[配置备份中，活让我干就好，您喝茶]

已登入设备：192.168.1.60
已获取配置
已保存配置
已登入设备：192.168.1.61
已获取配置
已保存配置
已登入设备：192.168.1.62
已获取配置
已保存配置
已登入设备：192.168.1.70
已获取配置
已保存配置
已登入设备：192.168.1.71
已获取配置
已保存配置
已登入设备：192.168.1.72
已获取配置
已保存配置
已登入设备：192.168.1.73
已获取配置
已保存配置
已登入设备：192.168.1.74
已获取配置
已保存配置
已登入设备：192.168.1.75
已获取配置
已保存配置

\(˙ω˙)==[备份结束，新备份文件已经就位，请大哥检查]
